### Import reviews

In [ ]:
%load_ext autoreload
%autoreload 2

from src.utils import log_progress as lp

In [ ]:
import pickle

DATA_FILE1 = 'data/data.pkl'
DATA_FILE2 = 'data/labels.pkl'
DATA_FILE3 = 'data/pos_adjectives_list.pkl'
DATA_FILE4 = 'data/neu_adjectives_list.pkl'
DATA_FILE5 = 'data/neg_adjectives_list.pkl'

with open(DATA_FILE1, 'rb') as f:
    reviews = pickle.load(f)
    
with open(DATA_FILE2, 'rb') as f:
    stars = pickle.load(f)

with open(DATA_FILE3, 'rb') as f:
    pos_adjectives_list = pickle.load(f)
    
with open(DATA_FILE4, 'rb') as f:
    neu_adjectives_list = pickle.load(f)

with open(DATA_FILE5, 'rb') as f:
    neg_adjectives_list = pickle.load(f)

In [ ]:
print(pos_adjectives_list)

In [ ]:
from babble.babbler import BabbleStream

babbler = BabbleStream(reviews,stars,balanced=True, shuffled=True, seed=456)

In [ ]:
babbler.semparser.grammar.lexical_rules

In [ ]:
from babble.utils import display_candidate

candidate = babbler.next()
print(candidate)
print(candidate.text)
print(candidate.entity_positions)

In [ ]:
print(dir(candidate.entities[0]))
print(candidate.entities)
print(candidate.entities.__getitem__(0))

In [ ]:
def check(candidate,adjective):
    for entity in candidate.entities:
        # qui controllo se gli aggettvi sono vicini alle entità...da capire bene come fare
        print(entity.entity)

In [ ]:
check(candidate,'fsdfsd')

In [ ]:
from babble import Explanation
explanation = Explanation(
        name='LF_1',
        label=1,
        condition='the word "my" is before X or the word "poor" is before X',
        candidate=candidate
    )

In [ ]:
babbler.view_parse(parses[0])

In [ ]:
print(parses[0].semantics)

In [ ]:
parses, filtered = babbler.apply(explanation)

In [ ]:
def create_condition(words):
    condition = 'the word '
    for word in words:
        condition = condition + '"'+ word + '" or '
    return condition + 'are in the sentence'

In [ ]:
import spacy

spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

print('spaCy Version: %s' % (spacy.__version__))
spacy_nlp = spacy.load('en_core_web_sm')

In [ ]:
from babble import Explanation

final_parses = []

index = 0

for element,label,selected_words in (zip(lp.log_progress(reviews[0]),stars[0],tokens_list)):
    doc = spacy_nlp(element.text)
    tokens = [token.text for token in doc if not token.is_stop and not token.pos_  in ('PUNCT', 'NUM', 'SYM')]
        
    print(selected_words)
    
    explanation = Explanation(
        name='LF_' + str(index),
        label=int(label),
        condition=create_condition(selected_words),
        candidate=element
    )
    parses, filtered = babbler.apply(explanation)
    babbler.commit()
    
    index = index + 1

In [ ]:
from metal.analysis import lf_summary

Ls = [babbler.get_label_matrix(split) for split in [0,1,2]]
lf_names = [lf.__name__ for lf in babbler.get_lfs()]
lf_summary(Ls[1], stars[1], lf_names=lf_names)

In [ ]:
from babble.utils import ExplanationIO

explanations = babbler.get_explanations()

FILE = "my_explanations.tsv"
exp_io = ExplanationIO()
exp_io.write(explanations, FILE)
explanations = exp_io.read(FILE)